In [5]:
%%html

Page Links — <a href="index.html" target="_self">Project Overview Page</a>
— <a href="page1.html" target="_self">Dataset Page</a>
— <a href="page2.html" target="_self">Data Imputation Page</a>
— <a href="page3.html" target="_self">Data Exploration Page</a>
— <a href="page4.html" target="_self">Data Modeling Page</a>

In [2]:
# Toggle raw code on/off, from stackoverflow
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle raw code on/off"></form>''')

# Upload Data

In [ ]:
import datetime
from sklearn.preprocessing import Imputer
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import random
import math
import pandas as pd
from sklearn.linear_model import LinearRegression as Lin_Reg
from statsmodels.regression.linear_model import OLS
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from dateutil.relativedelta import relativedelta
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeRegressor as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
import sklearn.preprocessing as Preprocessing
import csv
import seaborn as sns
from statsmodels.tsa.stattools import acf, pacf_ols, pacf
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.vector_ar.dynamic import DynamicVAR
import itertools as it
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
from pandas.tools.plotting import autocorrelation_plot
import matplotlib.colors as colors
import scipy as sp
from collections import Counter
import time
from statsmodels.graphics import utils
from scipy.stats import itemfreq
from itertools import combinations
%matplotlib inline
from sklearn.metrics import mean_squared_error
from math import sqrt

Since the train dataset is extremely large (> 25 megabytes), it cannot be uploaded directly to our Jupyter demo program. As a result, it was split into two separate files in excel, titled train_half_1.csv and train_half_2.csv. These were immediately merged once in Jupyter's environment. 

Per the Kaggle competition descriptions, the files represent the following:
- train_half_1.csv is 1st half of historical data including sales
- train_half_2.csv is 2nd half of historical data including sales
- test.csv is historical data excluding sales
- store.csv is supplemental information about the stores

The uploading process is featured below.

In [2]:
# upload data files
train_half_1 = pd.read_csv('train_half_1.csv')
train_half_2 = pd.read_csv('train_half_2.csv', header = None)
test = pd.read_csv('test.csv')
store = pd.read_csv('store.csv')

C:\Users\lukak\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
# view tails of data to double check file uploads
train_half_1.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
499993,144,3,4/9/14,6313,592,1,0,0,0
499994,145,3,4/9/14,6895,1115,1,0,0,0
499995,146,3,4/9/14,6562,672,1,0,0,0
499996,147,3,4/9/14,6669,825,1,0,0,0
499997,148,3,4/9/14,7225,757,1,0,0,0


In [42]:
train_half_2.tail()

,0,1,2,3,4,5,6,7,8
517206,1111,2,1/1/13,0,0,0,0,a,1
517207,1112,2,1/1/13,0,0,0,0,a,1
517208,1113,2,1/1/13,0,0,0,0,a,1
517209,1114,2,1/1/13,0,0,0,0,a,1
517210,1115,2,1/1/13,0,0,0,0,a,1


In [43]:
# rename columns
train_half_2.columns = ["Store", "DayOfWeek", "Date", "Sales", "Customers", "Open", "Promo", "StateHoliday", "SchoolHoliday"]

In [44]:
# check columns
train_half_2.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
517206,1111,2,1/1/13,0,0,0,0,a,1
517207,1112,2,1/1/13,0,0,0,0,a,1
517208,1113,2,1/1/13,0,0,0,0,a,1
517209,1114,2,1/1/13,0,0,0,0,a,1
517210,1115,2,1/1/13,0,0,0,0,a,1


In [45]:
test.tail()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0
41087,41088,1115,6,2015-08-01,1.0,0,0,1


In [46]:
store.tail()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN
1114,1115,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


In [47]:
# adjust data file for first half (corrupted)
train_half_1 = train_half_1.loc[0:499997, :]

# check if corrected
train_half_1.tail()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
499993,144,3,4/9/14,6313,592,1,0,0,0
499994,145,3,4/9/14,6895,1115,1,0,0,0
499995,146,3,4/9/14,6562,672,1,0,0,0
499996,147,3,4/9/14,6669,825,1,0,0,0
499997,148,3,4/9/14,7225,757,1,0,0,0


In [3]:
# append train_half_2 to the end of train_half_1
combined_set = [train_half_1, train_half_2]
train = pd.concat(combined_set)
print "There are " + str(np.shape(train)[0]) + " observations for the Rossman training dataset, which is what the Kaggle description describes. We may now proceed to the imputation phase of our analysis."

There are 1017209 observations for the Rossman training dataset, which is what the Kaggle description describes. We may now proceed to the imputation phase of our analysis.


In [6]:
%%html

Page Links — <a href="index.html" target="_self">Project Overview Page</a>
— <a href="page1.html" target="_self">Dataset Page</a>
— <a href="page2.html" target="_self">Data Imputation Page</a>
— <a href="page3.html" target="_self">Data Exploration Page</a>
— <a href="page4.html" target="_self">Data Modeling Page</a>